## Import package for data pipeline

In [24]:
import pandas as pd
import datetime

## Define function for cleaning date column

In [25]:
def clean_date_column(df, column_name):
    """
    Clean and format a date column to 'yyyy-mm-dd' format.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_clean = df.copy()
    
    # Function to convert individual values
    def convert_date(value):
        # Handle NaN, None, or empty strings
        if pd.isna(value) or value == '' or value is None:
            return value
        
        try:
            parsed_date = pd.to_datetime(value, errors='coerce')
            return parsed_date.strftime('%Y-%m-%d')
            
        except (ValueError, TypeError):
            # If any error occurs, return default
            return 
    
    # Apply the conversion function to the column
    df_clean[column_name] = df_clean[column_name].apply(convert_date)
    
    return df_clean

## Define function for calculation contract duration date

In [26]:
def calculate_contract_duration_days(df, start_col, end_col):
    """
    Calculate duration in days between start and end date columns.
    Only calculates if both dates are not null.
    
    """
    df_result = df.copy()
    
    def calc_days(row):
        start_date = row[start_col]
        end_date = row[end_col]

        # Skip if either date is None
        if start_date is None or end_date is None:
            return None
        
        try:
            # Convert to datetime and calculate difference
            start_dt = pd.to_datetime(start_date)
            end_dt = pd.to_datetime(end_date)
            return (end_dt - start_dt).days
        except:
            return None
    
    df_result['Contract Duration'] = df_result.apply(calc_days, axis=1)
    return df_result

## Define function filling null value for date column

In [27]:
def fill_null_dates(df, column_name, default_value='1900-01-01'):
    """
    Fill null values in a date column with a default value.

    """
    df_filled = df.copy()
    df_filled[column_name] = df_filled[column_name].fillna(default_value)
    return df_filled

## Read source csv data

In [28]:
df = pd.read_csv('CompetencyTest_Contracts_20250721.csv')

## Filling null value with Contract Type and Department columns.

In [29]:
df['Contract Type'] = df['Contract Type'].fillna('Unspecified Contract Type') # Fill missing contract type with default value "Unspecified Contract Type"
df['Department'] = df['Department'].fillna('Unspecified Department') # Fill missing department with default value "Unspecified Department"

## Format all date column to standard date yyyy-mm-dd

In [30]:
df = clean_date_column(df, 'Approval Date')  # Clean the 'Approval Date' column
df = clean_date_column(df, 'Start Date')  # Clean the 'Start Date' column
df = clean_date_column(df, 'End Date')  # Clean the 'End Date' column


## Calculate contract duration day

In [31]:
df = calculate_contract_duration_days(df, 'Start Date', 'End Date')  # Calculate contract duration in days

## Replace null value for dates columns.

In [32]:
df = fill_null_dates(df, 'Approval Date')  # Fill null value with default date in the 'Approval Date' column
df = fill_null_dates(df, 'Start Date')  # Fill null value with default date in the 'Start Date' column
df = fill_null_dates(df, 'End Date')  # Fill null value with default date in the 'End Date' column

## Check if it is Blanked Contract or not based on End date.

In [33]:
df['Is Blanket Contract'] = df['End Date'].apply(lambda x: 'Yes' if x is None else 'No')  # Create "Is Blanket Contract" column based on 'End Date'. If Contract End Date is None, then Yes, else No.

## Check if it is Legacy Contract or not based on first value of contract number.

In [34]:
df['Is Legacy Contract'] = df['Purchase Order (Contract) Number'].apply(lambda x: 'Yes' if str(x)[0].isalpha() else 'No') # Create "Is Legacy Contract" column based on 'Purchase Order (Contract) Number'. If the first character is an alphabet, then Yes, else No.

## Check if it Has Negative Modification or not based value of Award Amount.

In [35]:
df['Has Negative Modification'] = df['Award Amount'].apply(lambda x: 'Yes' if x < 0 else 'No')  # Create "Has Negative Modification" column based on 'Award Amount'. If Award Amount is negative, then Yes, else No.

## Save cleaned dataframe to csv file.

In [36]:
df.to_csv('cleaned_CompetencyTest_Contracts.csv', index=False)  # Save the cleaned DataFrame to a new CSV file